

*   pdfplumber 0.11.6
*   docx 1.1.2
*   langchain_community 0.3.23



In [ ]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 133.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 110.4 MB/s eta 0:00:00


In [ ]:
!pip install python-docx # install python-docx using pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 19.1 MB/s eta 0:00:00


In [ ]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 101.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.8 MB/s eta 0:00:00


In [ ]:
import langchain_community
print(langchain_community.__version__)

0.3.23


In [ ]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 109.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 100.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 128.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 5.6 MB/s eta

In [ ]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.0 MB/s eta 0:00:00


# Imports and Setup

In [ ]:
import os
import openai
import pdfplumber
import docx
import re
import json
import logging
from datetime import datetime
from bs4 import BeautifulSoup as Soup
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma


# Set up OpenAI

In [ ]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')


In [ ]:
client = openai.OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)

#  Extract and Clean CV Text

In [ ]:
def extract_text_from_pdf(file_path):
    text = ''
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + '\n'
    return text

def extract_text_from_docx(file_path):
    doc = docx.Document(file_path)
    return "\n".join(para.text for para in doc.paragraphs)

def clean_text(text):
    return re.sub(r'\s+', ' ', text).strip()

# Parse CV using GPT-4

In [ ]:
def parse_cv_with_openai(text):
    prompt = f"""
You are an expert resume parser.

Given the following CV text, extract the following information in JSON format:
- Skills (list)
- Education (list)
- Experience (list: mention years of experience and job titles if possible)

CV Text:
\"\"\"{text}\"\"\"

Return the JSON only, no extra text.
"""
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2
    )
    return response.choices[0].message.content


# Wrapper to Load and Parse CV

In [ ]:
def parse_cv(file_path, file_type='pdf'):
    if file_type == 'pdf':
        text = extract_text_from_pdf(file_path)
    elif file_type == 'docx':
        text = extract_text_from_docx(file_path)
    else:
        raise ValueError("Unsupported file type. Use PDF or DOCX.")

    text = clean_text(text)
    parsed_profile = parse_cv_with_openai(text)
    return parsed_profile, text


# Scrape and Cache Web Best Practices

In [ ]:
def get_or_create_vectorstore(url, persist_directory="./chroma_db"):
    index_path = os.path.join(persist_directory, "index")
    if os.path.exists(index_path):
        print(f"[INFO] Loading vectorstore from {persist_directory}")
        return Chroma(persist_directory=persist_directory, embedding_function=OpenAIEmbeddings())

    print("[INFO] Scraping and creating new vectorstore...")
    loader = RecursiveUrlLoader(
        url=url,
        max_depth=2,
        extractor=lambda x: Soup(x, "html.parser").text
    )
    docs = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
    splits = splitter.split_documents(docs)

    vectorstore = Chroma.from_documents(
        documents=splits,
        embedding=OpenAIEmbeddings(),
        persist_directory=persist_directory
    )

    return vectorstore


# Retrieve Best Practices for a Query

In [ ]:
def retrieve_best_practices(query, vectorstore, k=2):
    retriever = vectorstore.as_retriever()
    context = retriever.invoke(query,k=k)
    return " ".join([doc.page_content for doc in context])


# Answer Questions using RAG

In [ ]:
def generate_rag_answer(question, context, parsed_profile=None):
    prompt = f"""
You are an expert interview coach.

Use the following best practices and candidate profile to answer the question professionally.

Context (best practices): {context}

Candidate Profile (optional): {parsed_profile}

Interview Question: {question}

Give a strong example-based answer.
"""
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.5
    )
    return response.choices[0].message.content


# Generate Interview Questions

In [ ]:
def generate_interview_questions(parsed_profile, num_technical=5, num_behavioral=5):
    prompt = f"""
You are an expert HR recruiter.

Based on the following candidate profile, generate {num_technical} technical questions and {num_behavioral} behavioral questions.

Candidate Profile:
{parsed_profile}

Rules:
- Questions should be specific to the candidate's skills, education, and experience.
- Do not make up skills not mentioned.
- Return the result as a JSON with two fields: "technical_questions" and "behavioral_questions".
"""
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )
    return response.choices[0].message.content


# Full RAG Pipeline

In [ ]:
def run_rag_workflow(file_path, file_type='pdf', vectorstore=None):
    parsed_profile, raw_text = parse_cv(file_path, file_type)

    session_data = {
        "filename": os.path.basename(file_path),
        "uploaded_at": str(datetime.now()),
        "raw_text": raw_text,
        "parsed_profile": parsed_profile
    }

    questions_json = generate_interview_questions(session_data["parsed_profile"])
    question_dict = json.loads(questions_json)

    all_answers = {"technical_answers": [], "behavioral_answers": []}

    for q in question_dict["behavioral_questions"]:
        context = retrieve_best_practices(q, vectorstore)
        answer = generate_rag_answer(q, context, parsed_profile=session_data["parsed_profile"])
        all_answers["behavioral_answers"].append({"question": q, "answer": answer})

    for q in question_dict["technical_questions"]:
        context = retrieve_best_practices(q, vectorstore)
        answer = generate_rag_answer(q, context, parsed_profile=session_data["parsed_profile"])
        all_answers["technical_answers"].append({"question": q, "answer": answer})

    return all_answers, session_data


# Run Everything

In [ ]:
if __name__ == "__main__":
    file_path = "/content/RawanAlnajim_cv.pdf"  # Replace with actual path
    vectorstore_path = "./chroma_db"
    advice_url = "https://www.themuse.com/advice"

    vectorstore = get_or_create_vectorstore(advice_url, persist_directory=vectorstore_path)
    answers = run_rag_workflow(file_path, vectorstore=vectorstore)
    print(json.dumps(answers, indent=2))


[INFO] Scraping and creating new vectorstore...


[
  {
    "technical_answers": [
      {
        "question": "Can you explain a project where you used Python, SQL, CSS, and HTML together?",
        "answer": "Sure, I'd be happy to share an example. During my Data Science and Artificial Intelligence Bootcamp at T5 SDAIA, I was part of a team that worked on a project to develop a web-based application for sentiment analysis on social media posts.\n\nSituation: The goal of the project was to create an application that could analyze the sentiment of social media posts in real-time and present the results in an easily understandable format. \n\nTask: My task was to build the back-end of the application, which involved data collection, data processing, and implementing the sentiment analysis model. \n\nAction: I started by using Python to build a script that could scrape social media data in real-time. I used various Python libraries such as Tweepy for Twitter data and BeautifulSoup for web scraping. Once the data was collected, I used SQ

In [ ]:
answers, session_data = run_rag_workflow(file_path, vectorstore=vectorstore)

questions_json = generate_interview_questions(session_data["parsed_profile"])
question_dict = json.loads(questions_json)

print("🛠️ Technical Questions:")
for q in question_dict["technical_questions"]:
    print("-", q)

print("\n🧠 Behavioral Questions:")
for q in question_dict["behavioral_questions"]:
    print("-", q)


🛠️ Technical Questions:
- Can you explain a project where you utilized Python, SQL, CSS, and HTML together?
- How would you approach a data analysis task? Can you provide an example of a complex data analysis you've performed?
- Could you describe your experience with building ML, DL, and NLP models? What challenges did you face and how did you overcome them?
- Can you describe your proficiency with Microsoft Office, specifically Excel, Word, PowerPoint, and Power BI? How have you used these tools in your past roles?
- During your Data Science and Artificial Intelligence Bootcamp, what was the most challenging project you worked on and why?

🧠 Behavioral Questions:
- Can you provide an example of a time when you had to adapt quickly to a new situation or environment?
- Describe a situation where you had to work as part of a team. What was your role and how did you contribute to the team's success?
- How do you handle feedback and criticism? Can you provide an example?
- During your int